In [1]:
# import numpy as np
import pandas as pd

In [2]:
dfile = pd.read_csv("BMS Requirements.csv")
dfile

,Req ID,Req Name,Description,Pre-requisite,Cost,Value
0,101.0,URL,Unique web address to access the website,NaN,NaN,NaN
1,102.0,Choose Region Modal,To choose the city inorder to proceed,NaN,NaN,NaN
2,103.0,Home Page,Contains various...,102.0,NaN,NaN
3,104.0,Change Region,Enables to change the city,0.0,NaN,NaN
4,105.0,Search,To perform search,NaN,NaN,NaN
5,106.0,Search Results,Displays the search results,105.0,NaN,NaN
6,107.0,Login Button,Sign in to the system for existing users,NaN,NaN,NaN
7,108.0,Forgot Password,To enable sign in when user forgets password,NaN,NaN,NaN
8,109.0,Continue with Facebook,To login with Facebook account,NaN,NaN,NaN
9,110.0,Continue with Google,To login with Google account,NaN,NaN,NaN


In [3]:
dfile = dfile.drop(dfile.index[14:21])

In [4]:
dfile

,Req ID,Req Name,Description,Pre-requisite,Cost,Value
0,101.0,URL,Unique web address to access the website,NaN,NaN,NaN
1,102.0,Choose Region Modal,To choose the city inorder to proceed,NaN,NaN,NaN
2,103.0,Home Page,Contains various...,102.0,NaN,NaN
3,104.0,Change Region,Enables to change the city,0.0,NaN,NaN
4,105.0,Search,To perform search,NaN,NaN,NaN
5,106.0,Search Results,Displays the search results,105.0,NaN,NaN
6,107.0,Login Button,Sign in to the system for existing users,NaN,NaN,NaN
7,108.0,Forgot Password,To enable sign in when user forgets password,NaN,NaN,NaN
8,109.0,Continue with Facebook,To login with Facebook account,NaN,NaN,NaN
9,110.0,Continue with Google,To login with Google account,NaN,NaN,NaN


In [5]:
dfile['Req ID'] = (dfile['Req ID'].apply(str))
for i in range(dfile['Req ID'].count()):
    dfile['Req ID'][i] = str(dfile['Req ID'][i]).strip(".0")
dfile

/home/dmb/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Req ID,Req Name,Description,Pre-requisite,Cost,Value
0,101,URL,Unique web address to access the website,NaN,NaN,NaN
1,102,Choose Region Modal,To choose the city inorder to proceed,NaN,NaN,NaN
2,103,Home Page,Contains various...,102.0,NaN,NaN
3,104,Change Region,Enables to change the city,0.0,NaN,NaN
4,105,Search,To perform search,NaN,NaN,NaN
5,106,Search Results,Displays the search results,105.0,NaN,NaN
6,107,Login Button,Sign in to the system for existing users,NaN,NaN,NaN
7,108,Forgot Password,To enable sign in when user forgets password,NaN,NaN,NaN
8,109,Continue with Facebook,To login with Facebook account,NaN,NaN,NaN
9,11,Continue with Google,To login with Google account,NaN,NaN,NaN
